In [1]:
import os
from time import sleep

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import StaleElementReferenceException
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
driver = webdriver.Chrome('../chromedriver')
df = pd.read_csv('C:/rsource2/tour_data.csv')
df1 = df.loc[:,['tour_id','title']] 

In [3]:
df1.iloc[220:230]

,tour_id,title
220,221,국시집
221,222,국제갤러리
222,223,국제선센터
223,224,국제음식문화거리(INTERNATIONAL FOOD STREET)
224,225,국제자수원 3호점
225,226,국제청소년센터 유스호스텔
226,227,국화정원
227,228,국회 방문자센터 헌정기념관
228,229,국회의사당
229,230,굿모닝시티


In [4]:
title = [] #관광지이름
comments = [] #리뷰
ratings = [] #별점
tour_ids = [] #tour_id
write_date = [] #리뷰작성시기

In [5]:
def main():

    global driver, load_wb, review_num

    driver.implicitly_wait(3)  # 렌더링 될때까지 기다린다 3초
    driver.get('https://map.kakao.com/')  # 주소 가져오기

    for i in range(228,230):
        #delapy
        sleep(2)
        print('####',i+1)
        search(df1['title'][i],i+1)

    driver.quit()
    print("finish")

In [13]:
# 카카오맵 첫화면에서 관광지명으로 검색한뒤 첫번째 결과로 crawilng 함수 호출하는 함수

def search(place,tour_id):
    global driver

    search_area = driver.find_element_by_xpath('//*[@id="search.keyword.query"]')  # 검색 창
    search_area.send_keys(place)  # 검색어 입력
    driver.find_element_by_xpath('//*[@id="search.keyword.submit"]').send_keys(Keys.ENTER)  # Enter로 검색
    sleep(1)

    # 검색된 정보가 있는 경우에만 탐색
    # 1번 페이지 place list 읽기
    html = driver.page_source

    try:
        soup = BeautifulSoup(html, 'html.parser')
        place= soup.select('.placelist > .PlaceItem')[0] # 검색된 장소 목록
        try:
            sleep(1)
            crawling(place,tour_id)

        except ElementNotInteractableException:
            print('not found')
        finally:
            search_area.clear()
    except:
        print('no search result')
#         search_area.clear()

In [11]:
# 검색후 첫번째결과에 대해 상세정보를 새창으로 띄우는 함수

def crawling(place,tour_id):

        place_name = place.select('.head_item > .tit_name > .link_name')[0].text  # place name
        place_address = place.select('.info_item > .addr > p')[0].text  # place address

        detail_page_xpath = '//*[@id="info.search.place.list"]/li[1]/div[5]/div[4]/a[1]'
        driver.find_element_by_xpath(detail_page_xpath).send_keys(Keys.ENTER)
        driver.switch_to.window(driver.window_handles[-1])  # 상세정보 탭으로 변환
        sleep(1)

        print('####', place_name, tour_id)

        # 첫 페이지
        extract_review(place_name,tour_id)
        
        # 2-5 페이지
        idx = 3
        try:
            page_num = len(driver.find_elements_by_class_name('link_page')) # 페이지 수 찾기
            for i in range(page_num-1):
                # css selector를 이용해 페이지 버튼 누르기
                driver.find_element_by_css_selector('#mArticle > div.cont_evaluation > div.evaluation_review > div > a:nth-child(' + str(idx) +')').send_keys(Keys.ENTER)
                sleep(1)
                extract_review(place_name,tour_id)
                idx += 1
            driver.find_element_by_link_text('다음').send_keys(Keys.ENTER) # 5페이지가 넘는 경우 다음 버튼 누르기
            sleep(1)
            extract_review(place_name,tour_id) # 리뷰 추출
        except (NoSuchElementException, ElementNotInteractableException):
            print("no review in crawling")

        # 그 이후 페이지
        while True:
            idx = 4
            try:
                page_num = len(driver.find_elements_by_class_name('link_page'))
                for i in range(page_num-1):
                    driver.find_element_by_css_selector('#mArticle > div.cont_evaluation > div.evaluation_review > div > a:nth-child(' + str(idx) +')').send_keys(Keys.ENTER)
                    sleep(1)
                    extract_review(place_name,tour_id)
                    idx += 1
                driver.find_element_by_link_text('다음').send_keys(Keys.ENTER) # 10페이지 이상으로 넘어가기 위한 다음 버튼 클릭
                sleep(1)
                extract_review(place_name,tour_id) # 리뷰 추출
            except (NoSuchElementException, ElementNotInteractableException):
                print("no review in crawling")
                break

        driver.close()
        driver.switch_to.window(driver.window_handles[0])  # 검색 탭으로 전환

In [8]:
#상세정보창에서 리뷰내용과 별점, 작성시기를 크롤링하는 함수

def extract_review(place_name,tour_id):
    global driver

    ret = True

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    # 첫 페이지 리뷰 목록 찾기
    review_lists = soup.select('.list_evaluation > li')

    # 리뷰가 있는 경우
    if len(review_lists) != 0:
        for i, review in enumerate(review_lists):
            comment = review.select('.txt_comment > span') # 리뷰
            rating = review.select('.grade_star > em') # 별점
            date = review.select('.time_write')[0].text #작성날짜
            val = ''
            
            title.append(place_name)
            tour_ids.append(tour_id)
            write_date.append(date)
            
            if len(comment) != 0:
                if len(rating) != 0:
                    val = comment[0].text + '/' + rating[0].text.replace('점', '')
                else:
                    val = comment[0].text + '/0'

                comments.append(val)
                print(tour_id,val)

    #리뷰가 있고 여러페이지인경우..
    #해야됨
    else:
        print('no review in extract')
        ret = False

    return ret

In [14]:
if __name__ == "__main__":
    main()

#### 229
#### 국회의사당 229
229 이런 빨갱이보다 더한 인간들아 나라팔아먹지  말고 나라를 응원하고 화합 좀 합시다  더불어민주당화이팅 대박 수정원한다고  하루종일 전화해도 하루종일 통화중입니다  진짜 현실적인 복지를 위해서 풀어도 욕하는데  그건 경제 인플레이션을 막고 경제 침체를   회복시키기 위한 자구책임에도 부동산 활성  하면 내로남불이라 하고 인건비 올리면 상  권 움직이고 인건비.../5
229 진보국회 민주국회 세종으로 이전하자/5
229 전쟁나면 여기 의사당 뚜껑이 열리고 태권브이가 나와서 나라를 구한다. 그것만 아니었으면 진작에 국민들한테 불태워졌을 건물.... 정치하는 늙다리들 태권브이 믿고 너무 깝친다 일도 안 하고 맨날 싸우기나 하고... 한심한 노인네들./1
229 뭔 세종드립이야ㅋㅋㅋㅋㅋ/3
229 세종으로 좀 이전하라고;;/2
229 별루에요/1
229 정치좀 똑바로 해라/1
229 제발 저 혐오시설은 세종시로 가라/1
229 조금 아쉬워요/2
229 성매매특별법 만들고 경찰들 뇌물먹는 사건만 한가득  ㅋ 띨한  국회들 국회의원 성매매이슈  ㅋ경찰성매매이슈 법은 왜만들었냐?/1
229 별로예요/1
229 별로예요/1
229 별로예요/1
229 익스!!! 플로전!!!/1
229 ㅈ도 하는것도 없으면서 세금 빨아먹는 도둑놈들 /1
229 별로예요/1
229 서여의도 고도제한 좀 풀자.../1
229 별로예요/1
229 민주당도 개같지만 미통당은 그냥 답이없다 갱생도 불가능한 쓰레기당 동급으로 보기에도 더듬어 만진당한테 미안하다/1
229 좋아요/5
229 별로예요/1
229 쓰레기 집합소 국개의사당 여의도에 있는것도 맘에 안들어죽겠다 별 하나도 아깝다/1
229 제21대 국회 화이팅 더불어민주당 화이팅❗/5
229 여기 대깨문들이 점령해서 정치질 하네 ㅉㅉ /3
229 무단결석 해도 돈 받는 국회 여성가족부 다음으로 세금 아까운 집단/1
229 자유매국당 해체/1
229 최고예요/5
229 섞은정치인들 득실거리는 곳 세금

In [15]:
title

['국회의사당',
 '국회의사당',
 '국회의사당',
 '국회의사당',
 '국회의사당',
 '국회의사당',
 '국회의사당',
 '국회의사당',
 '국회의사당',
 '국회의사당',
 '국회의사당',
 '국회의사당',
 '국회의사당',
 '국회의사당',
 '국회의사당',
 '국회의사당',
 '국회의사당',
 '국회의사당',
 '국회의사당',
 '국회의사당',
 '국회의사당',
 '국회의사당',
 '국회의사당',
 '국회의사당',
 '국회의사당',
 '국회의사당',
 '국회의사당',
 '국회의사당',
 '국회의사당',
 '국회의사당',
 '국회의사당',
 '국회의사당',
 '국회의사당',
 '국회의사당',
 '국회의사당',
 '국회의사당',
 '국회의사당',
 '국회의사당',
 '국회의사당',
 '국회의사당',
 '국회의사당',
 '국회의사당',
 '국회의사당',
 '국회의사당',
 '국회의사당',
 '국회의사당',
 '국회의사당',
 '국회의사당',
 '국회의사당',
 '국회의사당',
 '국회의사당',
 '국회의사당',
 '국회의사당',
 '국회의사당',
 '국회의사당',
 '국회의사당',
 '국회의사당',
 '국회의사당',
 '국회의사당',
 '국회의사당',
 '국회의사당',
 '국회의사당',
 '국회의사당',
 '국회의사당',
 '국회의사당',
 '국회의사당',
 '국회의사당',
 '국회의사당',
 '국회의사당',
 '국회의사당',
 '국회의사당',
 '국회의사당',
 '국회의사당',
 '국회의사당',
 '국회의사당',
 '국회의사당',
 '국회의사당',
 '국회의사당',
 '국회의사당',
 '국회의사당',
 '국회의사당',
 '국회의사당',
 '국회의사당',
 '국회의사당',
 '국회의사당',
 '국회의사당',
 '국회의사당',
 '국회의사당',
 '국회의사당',
 '국회의사당',
 '국회의사당',
 '국회의사당',
 '국회의사당',
 '국회의사당',
 '국회의사당',
 '국회의사당',
 '국회의사당',
 '국회의사당',
 '국회의사당',
 '국회의사당',


In [16]:
for i in range(len(comments)):
    ratings.append(str(comments[i])[-1])
    comments[i] = comments[i][:-2]
    write_date[i] = write_date[i][:-1]

In [17]:
comments

['이런 빨갱이보다 더한 인간들아 나라팔아먹지  말고 나라를 응원하고 화합 좀 합시다  더불어민주당화이팅 대박 수정원한다고  하루종일 전화해도 하루종일 통화중입니다  진짜 현실적인 복지를 위해서 풀어도 욕하는데  그건 경제 인플레이션을 막고 경제 침체를   회복시키기 위한 자구책임에도 부동산 활성  하면 내로남불이라 하고 인건비 올리면 상  권 움직이고 인건비...',
 '진보국회 민주국회 세종으로 이전하자',
 '전쟁나면 여기 의사당 뚜껑이 열리고 태권브이가 나와서 나라를 구한다. 그것만 아니었으면 진작에 국민들한테 불태워졌을 건물.... 정치하는 늙다리들 태권브이 믿고 너무 깝친다 일도 안 하고 맨날 싸우기나 하고... 한심한 노인네들.',
 '뭔 세종드립이야ㅋㅋㅋㅋㅋ',
 '세종으로 좀 이전하라고;;',
 '이런 빨갱이보다 더한 인간들아 나라팔아먹지  말고 나라를 응원하고 화합 좀 합시다  더불어민주당화이팅 대박 수정원한다고  하루종일 전화해도 하루종일 통화중입니다  진짜 현실적인 복지를 위해서 풀어도 욕하는데  그건 경제 인플레이션을 막고 경제 침체를   회복시키기 위한 자구책임에도 부동산 활성  하면 내로남불이라 하고 인건비 올리면 상  권 움직이고 인건비...',
 '진보국회 민주국회 세종으로 이전하자',
 '전쟁나면 여기 의사당 뚜껑이 열리고 태권브이가 나와서 나라를 구한다. 그것만 아니었으면 진작에 국민들한테 불태워졌을 건물.... 정치하는 늙다리들 태권브이 믿고 너무 깝친다 일도 안 하고 맨날 싸우기나 하고... 한심한 노인네들.',
 '뭔 세종드립이야ㅋㅋㅋㅋㅋ',
 '세종으로 좀 이전하라고;;',
 '별루에요',
 '정치좀 똑바로 해라',
 '제발 저 혐오시설은 세종시로 가라',
 '조금 아쉬워요',
 '성매매특별법 만들고 경찰들 뇌물먹는 사건만 한가득  ㅋ 띨한  국회들 국회의원 성매매이슈  ㅋ경찰성매매이슈 법은 왜만들었냐?',
 '별로예요',
 '별로예요',
 '별로예요',
 '익스!!! 플로전!!!',
 'ㅈ도 하는것도 없으면서 세금 빨

In [18]:
temp1 = pd.DataFrame({'tour_id':tour_ids,'title':title,'comments':comments,'ratings':ratings,'date':write_date})

In [19]:
temp1

,tour_id,title,comments,ratings,date
0,229,국회의사당,이런 빨갱이보다 더한 인간들아 나라팔아먹지 말고 나라를 응원하고 화합 좀 합시다 ...,5,2020.10.31
1,229,국회의사당,진보국회 민주국회 세종으로 이전하자,5,2020.10.30
2,229,국회의사당,전쟁나면 여기 의사당 뚜껑이 열리고 태권브이가 나와서 나라를 구한다. 그것만 아니었...,1,2020.10.25
3,229,국회의사당,뭔 세종드립이야ㅋㅋㅋㅋㅋ,3,2020.10.24
4,229,국회의사당,세종으로 좀 이전하라고;;,2,2020.10.23
...,...,...,...,...,...
176,229,국회의사당,새누리당과 민주당 원내대표를 만나고 싶습니다.,5,2013.05.19
177,230,굿모닝시티,굿모닝 시티 일루션 테마월드 최고 각종VR체험,5,2019.08.29
178,230,굿모닝시티,"9층에 있는 메가박스에 영화 볼일 있어서 에스컬레이터 타고 올라가는데, 옷파는 3...",2,2018.06.04
179,230,굿모닝시티,손님한테 호객행위만하고 불쾌감만 준 추억을 가져다준 쇼핑몰이네요 잊혀지지 않습니다 ...,1,2014.08.11


In [22]:
a = temp1[temp1['tour_id']==229]

In [25]:
len(a)

177

In [ ]:
a.to_csv('kakaomap_review.csv',mode='w',encoding='utf8',index=False)